In [83]:
import pandas as pd
import numpy as np

In [84]:
df = pd.DataFrame(
    [
        ['Normal Ltd', 2018, 100],
        ['Normal Ltd', 2019, 200],
        ['Normal Ltd', 2020, 190],
        
        ['Founded Later Ltd', 2019, 190],
        ['Founded Later Ltd', 2020, 190],
        
        ['Closed Down Early Ltd', 2018, 100],
        ['Closed Down Early Ltd', 2019, 50],
        
        ['Negative Profits First Year Ltd', 2018, -100],
        ['Negative Profits First Year Ltd', 2019, 50],
        ['Negative Profits First Year Ltd', 2020, 150],
        
        ['Negative Profits Last Year Ltd', 2018, 50],
        ['Negative Profits Last Year Ltd', 2019, 150],
        ['Negative Profits Last Year Ltd', 2020, -100],
        
        ['Too Many Years Ltd', 2016, 25],
        ['Too Many Years Ltd', 2017, 25],
        ['Too Many Years Ltd', 2018, 50],
        ['Too Many Years Ltd', 2019, 40],
        ['Too Many Years Ltd', 2020, 30],
        
        ['Only One Year Ltd', 2020, 150],
        
        ['Started From Zero Ltd', 2018, 0],
        ['Started From Zero Ltd', 2019, 10],
        ['Started From Zero Ltd', 2020, 20],
    ],
    columns=['company_name', 'year', 'profit']
)

In [86]:
relevant_years = df[
    (df['year'] >= 2018) &
    (df['year'] <= 2020)]

In [97]:
idx_for_relevant_years = relevant_years.groupby(
    # Create a group.
    'company_name'
)['year'].agg(
    # For each company, get the index to the earliest (idxmin) 
    # and latest (idxmax) year
    ['idxmin', 'idxmax']
)
idx_for_relevant_years

,idxmin,idxmax
company_name,,
Closed Down Early Ltd,5,6
Founded Later Ltd,3,4
Negative Profits First Year Ltd,7,9
Negative Profits Last Year Ltd,10,12
Normal Ltd,0,2
Only One Year Ltd,18,18
Started From Zero Ltd,19,21
Too Many Years Ltd,15,17


In [98]:
first_years = relevant_years.loc[idx_for_relevant_years['idxmin']]
last_years = relevant_years.loc[idx_for_relevant_years['idxmax']]
last_years

,company_name,year,profit
6,Closed Down Early Ltd,2019,50
4,Founded Later Ltd,2020,190
9,Negative Profits First Year Ltd,2020,150
12,Negative Profits Last Year Ltd,2020,-100
2,Normal Ltd,2020,190
18,Only One Year Ltd,2020,150
21,Started From Zero Ltd,2020,20
17,Too Many Years Ltd,2020,30


In [89]:
merged_years = first_years.merge(
    last_years,
    # Merge the dataframes on the key "company_name"
    on='company_name',
    # When columns have the same name, apply these suffixes
    # to the column names so that we can keep them both
    suffixes=('_first', '_last')
)
merged_years

,company_name,year_first,profit_first,year_last,profit_last
0,Closed Down Early Ltd,2018,100,2019,50
1,Founded Later Ltd,2019,190,2020,190
2,Negative Profits First Year Ltd,2018,-100,2020,150
3,Negative Profits Last Year Ltd,2018,50,2020,-100
4,Normal Ltd,2018,100,2020,190
5,Only One Year Ltd,2020,150,2020,150
6,Started From Zero Ltd,2018,0,2020,20
7,Too Many Years Ltd,2018,50,2020,30


In [109]:
# All companies must have 2020 data.
# Use `.copy()` to copy the slice of the datframe and avoid warnings.
active_companies = merged_years.loc[merged_years['year_last'] == 2020].copy()

In [110]:
active_companies['year_diff'] = active_companies['year_last'] - active_companies['year_first']

In [111]:
import numpy as np
active_companies['cagr'] = np.power(
    # Profit of the last year divided by the first year profits
    active_companies['profit_last'] / active_companies['profit_first'],
    # Taken to the power of 1 divided by the time difference in years
    (1 / active_companies['year_diff'])
) - 1
active_companies

,company_name,year_first,profit_first,year_last,profit_last,year_diff,cagr
1,Founded Later Ltd,2019,190,2020,190,1,0.000000
2,Negative Profits First Year Ltd,2018,-100,2020,150,2,NaN
3,Negative Profits Last Year Ltd,2018,50,2020,-100,2,NaN
4,Normal Ltd,2018,100,2020,190,2,0.378405
5,Only One Year Ltd,2020,150,2020,150,0,0.000000
6,Started From Zero Ltd,2018,0,2020,20,2,inf
7,Too Many Years Ltd,2018,50,2020,30,2,-0.225403


In [115]:
def calculate_cagr(row):
    # If there is only one year, it is not possible to calculate CAGR.
    # As such, we return NaN.
    if row['year_diff'] == 0:
        return np.nan
    # Check if the first year profits is less or equal to 0 OR if
    # the last year profits is less than 0.
    # If so, handle the CAGR formula manually.
    if row['profit_first'] <= 0 or row['profit_last'] < 0:
        # If there was a growth in profits, return 'I' to represent 'Increase'
        if row['profit_first'] < row['profit_last']:
            return 'I'
        elif row['profit_first'] > row['profit_last']:
        # If there was a decline in profits, return 'D' to represent 'Decline'
            return 'D'
        # No difference in profits, so we return 0.
        else:
            return 0
    # If this row does not represent a special case, apply the CAGR formula
    return np.power(
        row['profit_last'] / row['profit_first'],
        (1 / row['year_diff'])
    ) - 1

In [116]:
active_companies['cagr_custom'] = active_companies.apply(
    # The function to apply, the function must take a 
    # row from the dataframe as an argument
    calculate_cagr,
    # Whether to apply the function for each column or for each row.
    # `axis=1` tells it to apply the function for each row.
    axis=1
)
active_companies

,company_name,year_first,profit_first,year_last,profit_last,year_diff,cagr,cagr_custom
1,Founded Later Ltd,2019,190,2020,190,1,0.000000,0.0
2,Negative Profits First Year Ltd,2018,-100,2020,150,2,NaN,I
3,Negative Profits Last Year Ltd,2018,50,2020,-100,2,NaN,D
4,Normal Ltd,2018,100,2020,190,2,0.378405,0.378405
5,Only One Year Ltd,2020,150,2020,150,0,0.000000,NaN
6,Started From Zero Ltd,2018,0,2020,20,2,inf,I
7,Too Many Years Ltd,2018,50,2020,30,2,-0.225403,-0.225403


In [139]:
%timeit merged_years.apply(calculate_cagr, axis=1)

657 µs ± 22.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [140]:
%timeit np.power(merged_years['profit_last'] / merged_years['profit_first'], (1 / merged_years['year_diff'])) - 1

401 µs ± 3.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
